#### Mayo 2024 
[Canción mas escuchada en Colombia en el 2018 🎼🎼](https://open.spotify.com/track/0VKkyBKCXyR99navhVRqcP?si=53cc15c7cbe04bac)
# ¿Cómo transformar un JSON + un CSV en una BBDD en python? 
### [Por: Carlos Eduardo Cortés Gomez](https://carloseduardo.omg.lol/)
-------------------------------------

El [Spotify Million Playlist Challenge](https://research.atspotify.com/2020/09/the-million-playlist-dataset-remastered/) es una estructura de datos interesantisima que contiene metadatos de un millón de listas creadas en Spotify + las canciones que la componen(con sus respectivos metadatos). El objetivo del conjunto de datos es crear un _sandbox_ de algoritmos, abierto al público, en el problema de sistemas de recomendación. ¿Que es un modelo de recomendación? Es el modelo que permite predecir que después de escuchar Peso Pluma en el 99% de los casos el usuario no quiere escuchar Justice, Judas Priest o Disclosure. Para nuestro curso es maravilloso para practicar técnicas de exploración y de análitca básica en conjuntos de datos estructurados.

- _Puede_ no ser representativa del comportamiento real de las listas en Spotify, pero probablemente tenga también información interesante
- _Tiene un problema_ pesa 5.6 gbs en json, lo cúal hace que no se pueda manipular directamente y dificulta el trabajo colaborativo

Este cuaderno tiene cómo objetivo transformar el json en un esquema SQL, filtrarla en un pedazo de interés y crear datos básicos sobre la BBDD

### Importante

> La BBDD completa no se va guardar en el repo, pero se puede descargar [aquí](https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge#challenge-dataset) (es necesario registro) para reemplazar en los strings donde se llama la base de datos.

## 

## Del formato JSON a un conjunto de Datos SQL

La base de datos en su formato RAW se ve así, es un JSON anidado en tres niveles:
1. Fecha y slice
2. Playlist info
3. Canciones de la Playlist

Vamos a crear tres esquemas que deberían funcionar así:

In [1]:
### Required Libraries
import json
import os
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import PrimaryKeyConstraint

Base = declarative_base()

In [2]:
# Define models
class SliceTime(Base):
    __tablename__ = 'slice_time'
    slice = Column(String, primary_key=True)
    generated_on = Column(String)
    version = Column(String)

class Playlists(Base):
    __tablename__ = 'playlists'
    slice = Column(String, ForeignKey('slice_time.slice'))
    pid = Column(Integer, primary_key=True)
    modified_at = Column(Integer)
    num_tracks = Column(Integer)
    num_albums = Column(Integer)
    num_followers = Column(Integer)
    num_edits = Column(Integer)        # New column for number of edits
    duration_ms = Column(Integer)      # New column for total duration in milliseconds
    num_artists = Column(Integer)      # New column for number of artists

class Song(Base):
    __tablename__ = 'song'
    track_uri = Column(String, nullable=False)
    pid = Column(Integer, ForeignKey('playlists.pid'), nullable=False)
    pos = Column(Integer)
    artist_name = Column(String)
    artist_uri = Column(String)
    track_name = Column(String)
    album_uri = Column(String)
    duration_ms = Column(Integer)
    album_name = Column(String)
    
    __table_args__ = (
        PrimaryKeyConstraint('track_uri', 'pid', 'pos'),
        {},
    )

In [3]:
# Load JSON data from a file
def load_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

In [4]:
# Set up the database connection and create tables
def setup_database(uri):
    engine = create_engine(uri)
    Base.metadata.create_all(engine)
    return engine

# Insert data into the database
def insert_data(session, data):
    # Insert into SliceTime
    info_data = data['info']
    slice_entry = SliceTime(
        generated_on=info_data['generated_on'], 
        slice=info_data['slice'],
        version=info_data['version']
    )
    session.add(slice_entry)
    
    # Insert into Playlists and Song
    for playlist in data['playlists']:
        playlist_entry = Playlists(
            slice=info_data['slice'],
            pid=playlist['pid'], 
            modified_at=playlist['modified_at'], 
            num_tracks=playlist['num_tracks'], 
            num_albums=playlist['num_albums'], 
            num_followers=playlist['num_followers'],
            num_edits=playlist['num_edits'],       # New data for number of edits
            duration_ms=playlist['duration_ms'],   # New data for total duration
            num_artists=playlist['num_artists']    # New data for number of artists
        )
        session.add(playlist_entry)
        
        for track in playlist['tracks']:
            track_entry = Song(
                pid=playlist['pid'], 
                pos=track['pos'], 
                artist_name=track['artist_name'], 
                artist_uri=track['artist_uri'], 
                track_uri=track['track_uri'], 
                track_name=track['track_name'], 
                album_uri=track['album_uri'], 
                duration_ms=track['duration_ms'], 
                album_name=track['album_name']
            )
            session.add(track_entry)
    
    session.commit()

def process_folder(folder_path):
    engine = setup_database('sqlite:///music.db')
    Session = sessionmaker(bind=engine)
    session = Session()
    
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            file_path = os.path.join(folder_path, filename)
            json_data = load_json(file_path)
            insert_data(session, json_data)
    
    session.close()

def main():
    folder_path = '/Users/carloscortes/Downloads/spotify_million_playlist_dataset(1)/data'
    process_folder(folder_path)

if __name__ == '__main__':
    main()

KeyboardInterrupt: 

In [ ]:
print("a")